In [1]:
import os

if "README.md" not in os.listdir():
    os.chdir("..")

print(os.listdir())

['sim_search.py', 'path_logP_QED.py', '.git', '__pycache__', 'GA_rediscover.py', 'stoned_selfies_tut.ipynb', 'generalized_path.py', 'notebooks', 'realize_path.py', 'environment.yml', 'readme_docs', 'data', 'README.md']


---

In [2]:
from pathlib import Path

In [3]:
import pandas as pd

df = pd.read_csv(
    Path("data") / "mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024.csv"
)
df.shape

(281634, 21)

In [4]:
filtered_by_fingerprints_df = df[
    ((df["ECFP6"] >= 0.5) & (df["ECFP6"] <= 0.8))
    | ((df["PATH"] >= 0.5) & (df["PATH"] <= 0.8))
]
filtered_by_fingerprints_df.shape

(7627, 21)

In [5]:
df_sorted = filtered_by_fingerprints_df.sort_values(by="syba_score", ascending=False)

In [6]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd


# Function to convert SMILES to 3D SDF
def smiles_to_sdf(smiles, output_filename):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol, AllChem.ETKDG())
    AllChem.UFFOptimizeMolecule(mol)
    writer = Chem.SDWriter(output_filename)
    writer.write(mol)
    writer.close()

    if os.path.exists(output_filename):
        os.remove(output_filename)


def check_sdf_save(smiles):
    output_filename = "temp.sdf"
    try:
        smiles_to_sdf(smiles, output_filename)
    except:
        return False
    return True

In [7]:
from typing import List
from tqdm import tqdm

In [8]:
valid_sdf_filter: List[bool] = []

count: int = 0

for row in tqdm(df_sorted.iterrows()):
    smiles = row[1]["canonical_smiles"]

    is_valid = check_sdf_save(smiles)
    if is_valid:
        valid_sdf_filter.append(pd.DataFrame([row[1]]))

    count += is_valid
    # print(f"COUNT: {count}")
    if count == 5000:
        break

2it [00:00, 14.59it/s][19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 18
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 18
5it [00:00, 21.33it/s][19:38:45] UFFTYPER: Unrecognized atom type: B_ (0)
[19:38:45] UFFTYPER: Unrecognized atom type: B_ (0)
11it [00:00, 24.33it/s][19:38:45] UFFTYPER: Unrecognized charge state for atom: 5
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 20
[19:38:45] UFFTYPER: Unrecognized charge state for atom: 5
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 20
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 1
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 19
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 19
14it [00:00, 24.10it/s][19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 16
[19:38:45] UFFTYPER: Warning: hybridization set to SP3 for atom 16
23it [00:00, 25.30it

In [9]:
df_final = pd.concat(valid_sdf_filter)

In [10]:
df_final.shape

(5000, 21)

In [11]:
df_final.to_csv(
    "mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024_5k_valid_sdf_filter.csv",
    index=False,
)

In [20]:
df_final[["molecule_id", "canonical_smiles"]].rename(
    {"canonical_smiles": "smiles"}, axis=1
).reset_index(drop=True).to_csv(
    "mutated_mol_df_deduplicated_with_syba_fp_sim-27.12.2024_5k_valid_sdf_filter_smiles_simplified.csv",
    index=False,
)